In [2]:
import ipytest
from dataclasses import dataclass

ipytest.autoconfig()

In [3]:
from pathlib import Path

In [4]:
data_path = Path.cwd() / 'data' / 'day4_input.txt'

In [5]:
data_path.exists()

True

#### Part 1

In [6]:
import numpy as np
import re

from collections.abc import Iterable
from itertools import groupby

In [7]:
def parse_grouped_strings_to_int_lists(grouped_string: Iterable[str]) -> list[int]:
    return [list(map(int, re.split(r'[,\s]+', line.strip()))) for line in grouped_string]
        

def read_arrays_from_file(file_path: Path) -> list[np.ndarray]:
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    arrays = [
        np.array(parse_grouped_strings_to_int_lists(group))
        for is_non_empty, group in groupby(lines, key=lambda x: bool(x.strip()))
        if is_non_empty
    ]
    
    return arrays

In [8]:
%%ipytest -vv

def test_parse_grouped_strings_to_int_lists():
    assert parse_grouped_strings_to_int_lists(['1, 2, 3\n', '4 5 6\n']) == [[1, 2, 3], [4, 5, 6]]
    assert parse_grouped_strings_to_int_lists(['   1, 2, 3  \n  ', '4 5 6\n']) == [[1, 2, 3], [4, 5, 6]]


======================================= test session starts ========================================
platform darwin -- Python 3.12.8, pytest-8.3.5, pluggy-1.5.0 -- /Users/hariravindran/Documents/workstation/Advent-of-Code-2021/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/hariravindran/Documents/workstation/Advent-of-Code-2021
configfile: pyproject.toml
collecting ... 

collected 1 item

t_53177e9c072b4b9198ed9e361e1b5898.py::test_parse_grouped_strings_to_int_lists PASSED        [100%]

======================================== 1 passed in 0.01s =========================================


In [9]:
%%ipytest -vv

def test_read_arrays_from_file(tmp_path):
    DATA_TEXT = """
    46,12,57,37,14,78,31,71,87,52,64,97,10,35,54,36

    37 72 60 35 89
    32 49  4 77 82
    """

    test_data_path = tmp_path / 'test_data.txt'
    test_data_path.write_text(DATA_TEXT)

    bingo_list_of_numbers, *bingo_boards = read_arrays_from_file(test_data_path)
    
    assert np.array_equal(bingo_list_of_numbers, np.array([[46, 12, 57, 37, 14, 78, 31, 71, 87, 52, 64, 97, 10, 35, 54, 36]]))
    assert np.array_equal(bingo_boards, [np.array([[37, 72, 60, 35, 89], [32, 49, 4, 77, 82]])])

======================================= test session starts ========================================
platform darwin -- Python 3.12.8, pytest-8.3.5, pluggy-1.5.0 -- /Users/hariravindran/Documents/workstation/Advent-of-Code-2021/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/hariravindran/Documents/workstation/Advent-of-Code-2021
configfile: pyproject.toml
collecting ... collected 1 item

t_53177e9c072b4b9198ed9e361e1b5898.py::test_read_arrays_from_file 

PASSED                     [100%]

======================================== 1 passed in 0.01s =========================================


In [10]:
bingo_array_of_numbers, *bingo_boards = read_arrays_from_file(data_path)

In [11]:
bingo_list_of_numbers = bingo_array_of_numbers.flatten().tolist()

In [12]:
bingo_boards[1]

array([[41, 94, 77, 43, 87],
       [ 2, 17, 82, 96, 25],
       [95, 49, 32, 12,  9],
       [59, 33, 67, 71, 64],
       [88, 54, 93, 85, 30]])

In [13]:
def check_win(mask: np.ndarray) -> bool:
    rows_win = np.sum(mask.all(axis=1))  # Count rows where all elements are True
    cols_win = np.sum(mask.all(axis=0))  # Count columns where all elements are True
    return (rows_win + cols_win) == 1

def calculate_score(winning_board: np.ndarray, drawn_numbers: list[int]) -> int:
    return np.sum(winning_board[~np.isin(winning_board, drawn_numbers)]) * drawn_numbers[-1]

In [14]:
%%ipytest -vv

def test_check_win_when_no_true_rows_or_columns():
    # Given
    mask = np.array([[True, True, True, False, False],
                     [False, False, False, False, False],
                     [False, False, False, False, False],
                     [False, False, False, False, False],
                     [False, False, False, True, True]])

    # When
    result = check_win(mask)

    # Then
    assert result == False


def test_check_win_when_one_true_row():
    # Given
    mask = np.array([[True, True, True, True, True],
                     [False, False, False, False, False],
                     [False, False, False, False, False],
                     [False, False, False, False, False],
                     [False, False, False, True, True]])

    # When
    result = check_win(mask)

    # Then
    assert result == True


def test_check_win_when_one_true_column():
    # Given
    mask = np.array([[True, False, False, False, False],
                     [True, False, False, False, False],
                     [True, False, False, False, False],
                     [True, False, False, False, False],
                     [True, True, True, True, False]])

    # When
    result = check_win(mask)

    # Then
    assert result == True


def test_check_win_when_true_column_and_true_row():
    # Given
    mask = np.array([[True, False, False, False, False],
                     [True, False, False, False, False],
                     [True, False, False, False, False],
                     [True, False, False, False, False],
                     [True, True, True, True, True]])

    # When
    result = check_win(mask)

    # Then
    assert result == False

======================================= test session starts ========================================
platform darwin -- Python 3.12.8, pytest-8.3.5, pluggy-1.5.0 -- /Users/hariravindran/Documents/workstation/Advent-of-Code-2021/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/hariravindran/Documents/workstation/Advent-of-Code-2021
configfile: pyproject.toml
collecting ... 

collected 4 items

t_53177e9c072b4b9198ed9e361e1b5898.py::test_check_win_when_no_true_rows_or_columns PASSED    [ 25%]
t_53177e9c072b4b9198ed9e361e1b5898.py::test_check_win_when_one_true_row PASSED               [ 50%]
t_53177e9c072b4b9198ed9e361e1b5898.py::test_check_win_when_one_true_column PASSED            [ 75%]
t_53177e9c072b4b9198ed9e361e1b5898.py::test_check_win_when_true_column_and_true_row PASSED   [100%]

======================================== 4 passed in 0.01s =========================================


In [15]:
%%ipytest -vv

def test_calculate_score():
    # Given
    winning_board = np.array([[1, 2, 3],
                               [4, 5, 6],
                               [7, 8, 9]])
    drawn_numbers = [1, 2, 3]

    # When
    result = calculate_score(winning_board, drawn_numbers)

    # Then
    assert result == (4 + 5 + 6 + 7 + 8 + 9) * 3

======================================= test session starts ========================================
platform darwin -- Python 3.12.8, pytest-8.3.5, pluggy-1.5.0 -- /Users/hariravindran/Documents/workstation/Advent-of-Code-2021/.venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/hariravindran/Documents/workstation/Advent-of-Code-2021
configfile: pyproject.toml
collecting ... collected 1 item

t_53177e9c072b4b9198ed9e361e1b5898.py::test_calculate_score PASSED                           [100%]



======================================== 1 passed in 0.01s =========================================


In [ ]:
# Instead of using multiple loops, forcing multiple breaks, we use a single loop.
# https://nedbatchelder.com/blog/201608/breaking_out_of_two_loops.html

for i in range(1, len(bingo_list_of_numbers) + 1):
    list_of_drawn_numbers = bingo_list_of_numbers[:i]
    winning_board = next((board for board in bingo_boards if check_win(np.isin(board, list_of_drawn_numbers))), None)
    if winning_board is not None:
        winning_score = calculate_score(winning_board=winning_board, drawn_numbers=list_of_drawn_numbers)
        print(winning_score)
        break

74320


#### Part 2

The issue here was modifying the loop above so that the first match doesn't result in a break, but instead saves the winning configuration and continues on to the next board. The change that needed to be made was that the list of boards then needed to be updated by removing the board in the winning configuration.

In [73]:
@dataclass
class WinningConfiguration:
    board: np.ndarray
    drawn_numbers: list[int]


winning_boards_and_numbers: list[WinningConfiguration] = []
for i in range(1, len(bingo_list_of_numbers) + 1):
    list_of_drawn_numbers = bingo_list_of_numbers[:i]
    winning_board = next((board for board in bingo_boards if check_win(np.isin(board, list_of_drawn_numbers))), None)
    if winning_board is not None:
        winning_boards_and_numbers.append(WinningConfiguration(board=winning_board, drawn_numbers=list_of_drawn_numbers))
        bingo_boards = [board for board in bingo_boards if not np.array_equal(board, winning_board)]
        if len(bingo_boards) == 0:
            break

In [74]:
len(winning_boards_and_numbers)

51

In [75]:
winning_boards_and_numbers[-1]

WinningConfiguration(board=array([[ 8, 34, 81, 67, 80],
       [83, 92, 13, 11, 41],
       [39, 89, 93, 49, 43],
       [20, 69,  3, 74, 76],
       [44, 72, 68, 70, 45]]), drawn_numbers=[46, 12, 57, 37, 14, 78, 31, 71, 87, 52, 64, 97, 10, 35, 54, 36, 27, 84, 80, 94, 99, 22, 0, 11, 30, 44, 86, 59, 66, 7, 90, 21, 51, 53, 92, 8, 76, 41, 39, 77, 42, 88, 29, 24, 60, 17, 68, 13, 79, 67, 50, 82, 25, 61, 20, 16, 6, 3, 81, 19, 85, 9, 28, 56, 75, 96, 2, 26, 1, 62, 33, 63, 32, 73, 18, 48, 43, 65, 98, 5, 91, 69, 47, 4, 38, 23, 49, 34])

In [76]:
calculate_score(
    winning_board=winning_boards_and_numbers[-1].board,
    drawn_numbers=winning_boards_and_numbers[-1].drawn_numbers
)

np.int64(17884)